In [1]:
import os
from tqdm import tqdm as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

In [2]:
torch.cuda.init()
device = torch.device('cuda:1')
print(torch.cuda.get_device_name(device))

GeForce GTX TITAN X


In [3]:
base_path = '/ssd2/chetanp'
batch_size = 2 ** 15

In [4]:
dataset = []
for i in range(2):
    path = os.path.join(base_path,'brown_e_base'+str(i+4)+'.pt')
    data = torch.load(path)
    dataset.append(TensorDataset(data))
dataset = ConcatDataset(dataset)
data_loader = DataLoader(dataset,batch_size = batch_size,num_workers = 14)

In [5]:
criterion = nn.CosineSimilarity()
loss_function = lambda x,y: (1-criterion(x,y)).mean()
val_losses = []
max_losses = []

In [6]:
print(next(iter(data_loader))[0].shape)

torch.Size([32768, 9216])


In [7]:
for i in range (7):
    
    class AutoEncoder(nn.Module):

        def __init__(self):
            super(AutoEncoder, self).__init__() 

            #encode
            self.e1 = nn.Linear(9216,2048)
            self.e2 = nn.Linear(2048,2 ** (5 + i))

            #decode
            self.d1 = nn.Linear(2 ** (5 + i),2048)
            self.d2 = nn.Linear(2048,9216)

        def forward(self, x):
            encode = self.e2(F.relu(self.e1(x)))

            return self.d2(F.relu(self.d1(encode)))
        
    model = nn.DataParallel(AutoEncoder(),device_ids =[1,2]).to(device)
    path = os.path.join(base_path,f'brown_e_base_compressor_{i}.pth')
    model.load_state_dict(torch.load(path))
    model.eval()
    
    running_loss = [0]*12
    count = [0]*12
    max_loss = [0]*12
    for data in tqdm(data_loader):
        ## Get Data
        sample = data[0].to(device)

        ## Pass forward
        output = model(sample)
        for j in range(12):
            loss = loss_function(output[:,j*768:(j+1)*768],sample[:,j*768:(j+1)*768])

            ## Update Running Loss
            max_loss[j] = max(loss.max().detach().item(), max_loss[j])
            running_loss[j] += float(loss)
            count[j] += 1
        
    running_loss = [l/c for l,c in zip(running_loss, count)]
    print(f'Test Set Loss {i}: ', str(running_loss))
    val_losses.append(running_loss)
    print(f'Max Loss {i}: ', max_loss)
    max_losses.append(max_loss)

100%|██████████| 9/9 [00:14<00:00,  1.12s/it]


Test Set Loss 0:  [0.17332986659473842, 0.1680613938305113, 0.16893990668985578, 0.16928874949614206, 0.17065892120202383, 0.18725264900260502, 0.2062772942913903, 0.21521369450622135, 0.23796693483988443, 0.25154490437772536, 0.23324895898501077, 0.24688736763265398]
Max Loss 0:  [0.18103119730949402, 0.1751817762851715, 0.1750163435935974, 0.17467601597309113, 0.17563968896865845, 0.19198527932167053, 0.21141797304153442, 0.2208268940448761, 0.24429336190223694, 0.2593735456466675, 0.240241140127182, 0.25391513109207153]


100%|██████████| 9/9 [00:11<00:00,  1.05it/s]


Test Set Loss 1:  [0.1557164423995548, 0.14829307132297093, 0.14767450590928397, 0.14609264334042868, 0.14616353644265068, 0.15994197461340162, 0.176122784614563, 0.18313850296868217, 0.20288185444143084, 0.21636546651522318, 0.2036844657527076, 0.2174583011203342]
Max Loss 1:  [0.16280311346054077, 0.15479959547519684, 0.15326747298240662, 0.1511506736278534, 0.15087342262268066, 0.1643928438425064, 0.18102425336837769, 0.18852654099464417, 0.20898032188415527, 0.22378772497177124, 0.2104792594909668, 0.22462481260299683]


100%|██████████| 9/9 [00:11<00:00,  1.05it/s]


...

Test Set Loss 4:  [0.1053767560256852, 0.09215384059482151, 0.08477670285436842, 0.07698223491509755, 0.07295451230472988, 0.0770069162050883, 0.08360537389914195, 0.08686482906341553, 0.09794507920742035, 0.11389868375327852, 0.11717892355389065, 0.1317838504910469]
Max Loss 4:  [0.10939471423625946, 0.09542344510555267, 0.08727289736270905, 0.07892236113548279, 0.07460374385118484, 0.07862542569637299, 0.08546369522809982, 0.08894728124141693, 0.1008758395910263, 0.1181252971291542, 0.12150555849075317, 0.13664984703063965]


100%|██████████| 9/9 [00:11<00:00,  1.03it/s]


Test Set Loss 5:  [0.06926281253496806, 0.0551676530804899, 0.048691672583421074, 0.0446122106578615, 0.04385523870587349, 0.04654402037461599, 0.0500619113445282, 0.05031870802243551, 0.05395748631821738, 0.06494389474391937, 0.07296013252602683, 0.08945830745829476]
Max Loss 5:  [0.07185392081737518, 0.05695053189992905, 0.04987415671348572, 0.04548874497413635, 0.04468629136681557, 0.04755764827132225, 0.051209405064582825, 0.05156930536031723, 0.05548346787691116, 0.06721983850002289, 0.07546535134315491, 0.09275596588850021]


100%|██████████| 9/9 [00:11<00:00,  1.00s/it]

Test Set Loss 6:  [0.05357623886730936, 0.03962654537624783, 0.036833407978216805, 0.03621047486861547, 0.037107445713546544, 0.03940820652577612, 0.04145840969350603, 0.041335412611564, 0.04279564983314938, 0.04898795485496521, 0.0550538988576995, 0.07299298875861698]
Max Loss 6:  [0.05547547712922096, 0.0407819002866745, 0.0376272052526474, 0.036916181445121765, 0.03785283863544464, 0.04030955955386162, 0.04244134575128555, 0.04239330813288689, 0.04405728727579117, 0.05058640241622925, 0.056825682520866394, 0.07539293169975281]


In [8]:
import csv

path = os.path.join(base_path, 'val_loss_layerwise.csv')

with open(path, 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerows(val_losses)
    wr.writerows(max_losses)